In [1]:
#magic code
%matplotlib inline

#import pathlib
import imageio
import numpy as np
import matplotlib.pyplot as plt
import skimage

# Glob the training data and load a single image path
#training_paths = pathlib.Path('/modules/cs342/Assignment2/FullTraining').glob('*/images/*.png')
#training_sorted = sorted([x for x in training_paths])
#im_path = training_sorted[0]
#im = imageio.imread(str(im_path))
#plt.imshow(im)

#Import an image
im = imageio.imread('C:/Users/Joe/Downloads/stage1/stage1_train/00ae65c1c6631ae6f2be1a449902976e6eb8483bf6b0740d00530220832c6d3e/images/00ae65c1c6631ae6f2be1a449902976e6eb8483bf6b0740d00530220832c6d3e.png')
plt.imshow(im)
#
# Convert to gray
from skimage.color import rgb2gray
im_gray = rgb2gray(im)

#Remove the background by otsu method
from skimage.filters import threshold_otsu
thresh_val = threshold_otsu(im_gray)
mask = np.where(im_gray > thresh_val, 1, 0)
# Make sure the larger portion of the mask is considered background
if np.sum(mask==0) < np.sum(mask==1):
    mask = np.where(mask, 0, 1)
    
# Deriving individual mask for each object
from scipy import ndimage
labels, nlabels = ndimage.label(mask)
label_arrays = []
for label_num in range(1, nlabels+1):
    label_mask = np.where(labels == label_num, 1, 0)
    print(sum(sum(label_mask)))
    label_arrays.append(label_mask)
    
#Removing small (fake) cells
for label_ind, label_coords in enumerate(ndimage.find_objects(labels)):
    cell = im_gray[label_coords]
    # Check if the label size is too small
    if np.product(cell.shape) < 10: 
        mask = np.where(labels==label_ind+1, 0, mask)
        print('Label {} is too small! Setting to 0.'.format(label_ind))
# Regenerate the labels and mask
labels, nlabels = ndimage.label(mask)
#for label_num in range(1, nlabels+1):
#    label_mask = np.where(labels == label_num, 1, 0)
#    label_arrays.append(label_mask)

    
#RLE Encoding
print('RLE Encoding for the current mask is: {}'.format(rle_encoding(mask)))

IOError: No such file: 'C:\Users\Joe\Downloads\stage1\stage1_train\00ae65c1c6631ae6f2be1a449902976e6eb8483bf6b0740d00530220832c6d3e\images\00ae65c1c6631ae6f2be1a449902976e6eb8483bf6b0740d00530220832c6d3e.png'

In [ ]:
def rle_encoding(x):
    '''
    x: numpy array of shape (height, width), 1 - mask, 0 - background
    Returns run length as list
    '''
    dots = np.where(x.T.flatten()==1)[0] # .T sets Fortran order down-then-right
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b+1, 0))
        run_lengths[-1] += 1
        prev = b
    return " ".join([str(i) for i in run_lengths])

In [ ]:
# GETTING A SINGLE IMAGE
#magic code
%matplotlib inline
import pandas as pd
from skimage.filters import threshold_otsu
from scipy import ndimage
import pathlib
import imageio
import numpy as np
import matplotlib.pyplot as plt
from skimage.color import rgb2gray
from skimage.filters import threshold_otsu
from scipy import ndimage


def rle_encoding(x):
    '''
    x: numpy array of shape (height, width), 1 - mask, 0 - background
    Returns run length as list
    '''
    dots = np.where(x.T.flatten()==1)[0] # .T sets Fortran order down-then-right
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b+1, 0))
        run_lengths[-1] += 1
        prev = b
    return " ".join([str(i) for i in run_lengths])



def RLE_mask(image,image_id,df):
    '''
    Returns pandas dataframe of each RLE string for each (predicted) nuclei of an image
    '''
    # Convert to gray
    im_gray = rgb2gray(image)

    #Remove the background by otsu method
    thresh = threshold_otsu(im_gray)
    mask = np.where(im_gray > thresh, 1, 0)
    # Make sure the larger portion of the mask is considered background
    if np.sum(mask==0) < np.sum(mask==1):
        mask = np.where(mask, 0, 1)

    # Deriving individual mask for each object
    labels, numlabels = ndimage.label(mask)     #labels each component with different number
    label_arrays = []
    im_df = pd.DataFrame(columns=["ImageID","EncodedPixels"])
    for n in range(1, numlabels+1, 1):
        label_mask = np.where(labels == n, 1, 0)
        if sum(sum(label_mask))>=10:        # checks if the nuclei is big enough to be considered
            label_arrays.append(label_mask)
        else:
            mask = np.where(labels == n, 0, mask)
            numlabels += -1
    labels, emptyarg = ndimage.label(mask)

    # Watershed by distance to edge
    #distance = ndi.distance_transform_edt(labels)
    #local_maxi = peak_local_max(distance, indices=False, labels=labels)
    #markers = ndi.label(local_maxi)[0]
    #labels = watershed(-distance, markers, mask=labels)

    #Adding to df
    im_df = pd.DataFrame(columns=["ImageID","EncodedPixels"])
    for n in range(1, numlabels+1, 1):
            label_mask = np.where(labels == n, 1, 0)
            rle_string = rle_encoding(label_mask)
            series = pd.Series({'ImageID': image_id, 'EncodedPixels': rle_string})
            im_df = im_df.append(series, ignore_index=True)

    return im_df


# MAIN

#import pathlib
import imageio
import numpy as np
import matplotlib.pyplot as plt
import skimage
from scipy import ndimage as ndi
from skimage.morphology import watershed
from skimage.feature import peak_local_max


# This will be our final dataframe
df = pd.DataFrame(columns=["ImageID","EncodedPixels"])

# Glob the training data and load a single image path
training_paths = pathlib.Path('/modules/cs342/Assignment2/FullTesting/').glob('*/images/*.png')
training_sorted = sorted([x for x in training_paths])
for image_path in training_sorted:
    image_id = image_path.parts[-3]
    image = imageio.imread(str(image_path))
    add_to_df = RLE_mask(image,image_id,df)
    df = df.append(add_to_df, ignore_index=True)

    
df.to_csv(path_or_buf='../submission.csv', index=None)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
from skimage.morphology import watershed
from skimage.feature import peak_local_max

#Getting an image
training_paths = pathlib.Path('/modules/cs342/Assignment2/FullTraining').glob('*/images/*.png')
training_sorted = sorted([x for x in training_paths])
im_path = training_sorted[0]
image = imageio.imread(str(im_path))

